# 1. Import thư viện

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    datasets \
    evaluate \
    rouge_score \
    loralib \
    peft --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.1 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

2024-08-21 02:50:36.047633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 02:50:36.047737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 02:50:36.220316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 2. Preprocess Dataset

In [3]:
dataset = load_dataset("toanduc/t5-sumary-dataset") #  vietgpt/news_summarization_vi

Generating train split:   0%|          | 0/9943 [00:00<?, ? examples/s]

# 3. Load LLM

In [4]:
tokenizer = AutoTokenizer.from_pretrained("pengold/t5-vietnamese-summarization")
original_model = AutoModelForSeq2SeqLM.from_pretrained("pengold/t5-vietnamese-summarization")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
def number_of_parameters(model):
    num_parameters = 0
    num_trainable_parameters = 0
    for name, param in model.named_parameters():
        num_parameters += param.numel()
        if param.requires_grad:
            num_trainable_parameters += param.numel()
    print(f'Số lượng tham số trong mô hình: {num_parameters}')
    print(f'Số lượng tham số có thể huấn luyện: {num_trainable_parameters}')
    print(f'Phần trăm tham số huấn luyện: {100*num_trainable_parameters/num_parameters}%')

In [6]:
number_of_parameters(original_model)

Số lượng tham số trong mô hình: 60506624
Số lượng tham số có thể huấn luyện: 60506624
Phần trăm tham số huấn luyện: 100.0%


**Kiểm tra mô hình với zero shot**

In [7]:
text = '''
Suốt bao năm, để dòng tranh này không bị rơi vào quên lãng, mỗi ngày người ta đều thấy ông Đạt cặm cụi làm nên những bức tranh từ mũi dao, cán đục. Ông bảo, tranh sơn khắc ở nước ta ra đời sớm nhất và còn đẹp hơn cả tranh sơn khắc của Nhật. Quý giá như vậy nên ông chẳng thể để nghề mai một trong sự chông chênh của thời cuộc. Một trong những sản phẩm sơn khắc của ông Đạt được trả 25 triệu. Theo ông Đạt, thời điểm năm 1945 đến 1995 là lúc tranh sơn khắc ở nước ta phát triển mạnh nhất. Thời điểm đó, các sản phẩm của Hạ Thái chiếm tới 70% hàng xuất khẩu, giải quyết được công ăn việc làm cho người dân trong làng và cả các địa phương khác, đem lại cuộc sống khấm khá cho nhiều hộ gia đình. Say mê hội họa từ nhỏ, nên chuyện ông Đạt đến với tranh sơn khắc như một mối duyên tiền định. Khi mới tiếp xúc với những bức tranh này, ông Đạt như bị lôi cuốn chẳng thể nào dứt ra được. Học hết cấp 3, tôi thi vào Đại học sư phạm nhưng sức khỏe không đảm bảo nên xin vào làm thợ vẽ trong xưởng của hợp tác xã. Năm 1979, tôi được hợp tác xã cử đi học thêm ở trường Mỹ Nghệ. Khi về lại xưởng, nhờ năng khiếu hội họa nên tôi được chuyển sang khâu đoạn khảm trai rồi sang tranh khắc. Tôi làm tranh khắc từ đó đến giờ ông Đạt chia sẻ. Theo lời ông Đạt, học sơn khắc khó bởi cách vẽ của dòng tranh này khác hẳn với sơn mài. Nếu như sơn mài người ta có thể vẽ bằng chổi hay bút lông, cũng có khi là chất liệu mềm rồi mới quét sơn lên vóc thì sơn khắc khâu đoạn lại làm khác hẳn. Sơn khắc là nghệ thuật của đồ họa, sự hoàn thiện của bức tranh phụ thuộc vào những nét chạm khắc và những mảng hình tinh tế, giàu cảm xúc. Cuối cùng mới là việc tô màu nhằm tạo sự khắc họa mạnh. Như một lẽ xoay vần tự nhiên, sự phát triển của làng nghề Hạ Thái dần chùng xuống. Làng nghề bước vào thời kỳ suy thoái, đặc biệt là trong giai đoạn khủng hoảng kinh tế Đông Âu từ 1984 đến 1990 đã làm hợp tác xã tan rã. Ông Đạt khi đó cũng như bao người thợ khác đều phải quay về làm ruộng. Ông Đạt giải thích, tranh sơn khắc xuất phát từ gốc tranh sơn mài. Nếu như ở tranh sơn mài thông thường, để có một tấm vóc vẽ người ta phủ sơn ta, vải lên tấm gỗ và mài phẳng thì tranh sơn khắc độc đáo ở chỗ, phải sử dụng kỹ thuật thủ công để khắc lên tấm vóc sơn mài. Tranh sơn khắc từ phôi thai, phác thảo đến lúc hoàn thành có khi kéo dài cả năm trời. Chẳng hạn, riêng công khắc ở bức tranh khổ nhỏ thường tôi làm cả ngày lẫn đêm thì mất 2 ngày, phối màu mất 3 ngày. Để người trẻ học được nghề cũng sẽ mất khoảng 6 tháng đến 1 năm - ông Trần Thành Đạt chia sẻ. Tranh sơn khắc đòi hỏi rất kỹ về phác thảo, bố cục, cũng như mảng màu sáng tối mà màu đen của vóc là chủ đạo. Dù trên diện tích bức tranh khổ lớn bao nhiêu nó vẫn rất cần kỹ càng và chính xác đến từng xen-ti-met. Nếu sai, bức tranh sẽ gần như bị hỏng, các đường nét phải khắc họa lại từ đầu. Kỳ công là vậy nên giá thành mỗi sản phẩm sơn khắc thường khá cao, trung bình từ 4 đến 25 triệu đồng/bức tranh. Giá thành cao lại yêu cầu khắt khe về mặt kỹ thuật, mỹ thuật nên theo Nghệ nhân Trần Thành Đạt, nhiều người trong làng đã từ bỏ, không làm dòng tranh này nữa. Tranh sơn khắc làm mất nhiều thời gian và công sức nhưng khó bán. Họ đều tập trung làm tranh sơn mài, với chất liệu ngoại nhập cho rẻ và ít tốn công sức. Hầu như cả làng đã quay lưng, bỏ rơi dòng tranh sơn khắc vào lãng quên ông Đạt buồn bã kể. Được biết, hiện xưởng sản xuất tranh của ông Đạt chủ yếu là các thành viên trong gia đình. Ông khoe, hai con trai và con gái đều tốt nghiệp Trường Đại học Mĩ thuật, con rể và các con dâu cũng là họa sĩ của trường. Tất cả các thành viên trong gia đình ông đều chung niềm say mê với sơn khắc. Đinh Luyện.
'''

In [8]:
prompt = f"""

Summarize the following text:

Text:
{text}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=1000,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(f'INPUT PROMPT:\n{prompt}')
print(output)

Token indices sequence length is longer than the specified maximum sequence length for this model (882 > 512). Running this sequence through the model will result in indexing errors


INPUT PROMPT:


Summarize the following text:

Text:

Suốt bao năm, để dòng tranh này không bị rơi vào quên lãng, mỗi ngày người ta đều thấy ông Đạt cặm cụi làm nên những bức tranh từ mũi dao, cán đục. Ông bảo, tranh sơn khắc ở nước ta ra đời sớm nhất và còn đẹp hơn cả tranh sơn khắc của Nhật. Quý giá như vậy nên ông chẳng thể để nghề mai một trong sự chông chênh của thời cuộc. Một trong những sản phẩm sơn khắc của ông Đạt được trả 25 triệu. Theo ông Đạt, thời điểm năm 1945 đến 1995 là lúc tranh sơn khắc ở nước ta phát triển mạnh nhất. Thời điểm đó, các sản phẩm của Hạ Thái chiếm tới 70% hàng xuất khẩu, giải quyết được công ăn việc làm cho người dân trong làng và cả các địa phương khác, đem lại cuộc sống khấm khá cho nhiều hộ gia đình. Say mê hội họa từ nhỏ, nên chuyện ông Đạt đến với tranh sơn khắc như một mối duyên tiền định. Khi mới tiếp xúc với những bức tranh này, ông Đạt như bị lôi cuốn chẳng thể nào dứt ra được. Học hết cấp 3, tôi thi vào Đại học sư phạm nhưng sức khỏe không đảm

**Kiểm tra mô hình với one shot**



In [9]:
prompt = f"""
Summarize the following text:

Text:
Thủ tướng Nguyễn Xuân Phúc và Bộ trưởng Ngoại giao Nhật Bản Taro Kono - Ảnh: VGP/Quang Hiếu. Tại buổi tiếp,
Thủ tướng Nguyễn Xuân Phúc bày tỏ vui mừng trước sự phát triển nhanh chóng, thực chất và toàn diện của quan
hệ Đối tác chiến lược sâu rộng Việt Nam - Nhật Bản; cho rằng việc hai nước đẩy mạnh trao đổi và tiếp xúc 
đoàn cấp cao thời gian qua đã giúp tăng cường sự tin cậy- điều kiện quan trọng để thúc đẩy quan hệ Việt 
Nam - Nhật Bản phát triển hơn nữa trên mọi lĩnh vực; mong muốn tăng cường hợp tác ASEAN - Nhật Bản và 
Mekong - Nhật Bản, nhất là khi Việt Nam đảm nhiệm vai trò Điều phối viên quan hệ ASEAN - Nhật Bản giai 
đoạn 2018 - 2021; đánh giá cao Nhật Bản đã cung cấp vốn hỗ trợ phát triển chính thức ODA cho Việt Nam thời
gian qua, khẳng định Chính phủ Việt Nam coi trọng và nỗ lực thực hiện hiệu quả các dự án vốn vay ODA giữa
hai nước. Ảnh: VGP/Quang Hiếu. Thủ tướng Nguyễn Xuân Phúc đề nghị Bộ trưởng Ngoại giao Nhật Bản quan tâm 
thúc đẩy liên kết kinh tế giữa 2 nước, đặc biệt là những lĩnh vực có nhiều tiềm năng như hợp tác thương 
mại, đầu tư, đào tạo nguồn nhân lực; tiếp tục cung cấp vốn vay ODA cho Việt Nam trong các lĩnh vực xây 
dựng cơ sở hạ tầng, đào tạo nguồn nhân lực, ứng phó với biến đổi khí hậu; thúc đẩy hợp tác trong lĩnh vực 
cải cách hành chính, hỗ trợ Việt Nam xây dựng chính phủ điện tử; phối hợp thực hiện Hiệp định Đối tác 
Toàn diện và Tiến bộ xuyên Thái Bình Dương (CPTPP) và trong đàm phán Hiệp định Đối tác kinh tế khu vực 
(RCEP); tăng cường tiếp nhận lao động Việt Nam, thúc đẩy giao lưu nhân dân và cùng phối hợp chặt chẽ trong
các vấn đề quốc tế và khu vực thời gian tới. Thủ tướng Nguyễn Xuân Phúc cũng đánh giá cao các hoạt động 
kỷ niệm có ý nghĩa đang diễn ra tại hai nước nhân dịp kỷ niệm 45 năm thiết lập quan hệ ngoại giao Việt 
Nam - Nhật Bản, góp phần quan trọng vào tăng cường hiểu biết, thúc đẩy giao lưu nhân dân giữa hai nước. 
Ảnh: VGP/Quang Hiếu. Bộ trưởng Taro Kono bày tỏ vui mừng được quay lại Hà Nội sau 27 năm, đúng vào dịp kỷ niệm 45 năm thiết lập quan hệ ngoại giao và tham dự các hoạt động của Hội nghị WEF ASEAN tại Việt Nam; bày tỏ mong muốn đóng góp vào thành công của Hội nghị và thúc đẩy hơn nữa quan hệ hợp tác Việt Nam - Nhật Bản. Bộ trưởng khẳng định Nhật Bản đánh giá cao vai trò ngày càng chủ động và đóng góp tích cực của Việt Nam trong các vấn đề quốc tế và khu vực; cảm ơn việc Thủ tướng sẽ dự Hội nghị Cấp cao Mekong - Nhật Bản thời gian tới theo lời mời của Chính phủ Nhật Bản, đề nghị hai bên phối hợp chặt chẽ vì thành công của Hội nghị. Bộ trưởng Taro Kono khẳng định Nhật Bản sẽ hỗ trợ Việt Nam nâng cao năng suất lao động, chia sẻ kinh nghiệm trong xây dựng Chính phủ điện tử; ủng hộ quan điểm của Việt Nam và các nước ASEAN trong vấn đề Biển Đông khi Việt Nam đảm nhiệm vai trò Điều phối viên Quan hệ ASEAN - Nhật Bản từ năm 2018. Đức Tuân.

Summary:
Thủ tướng Nguyễn Xuân Phúc và Bộ trưởng Ngoại giao Nhật Bản Taro Kono đã có buổi tiếp để thảo luận về việc
phát triển quan hệ Đối tác chiến lược sâu rộng giữa Việt Nam và Nhật Bản. Thủ tướng Nguyễn Xuân Phúc đề 
nghị Bộ trưởng Ngoại giao Nhật Bản quan tâm thúc đẩy liên kết kinh tế giữa 2 nước, cung cấp vốn vay ODA 
cho Việt Nam, tăng cường tiếp nhận lao động Việt Nam và cùng phối hợp chặt chẽ trong các vấn đề quốc tế 
và khu vực. Bộ trưởng Taro Kono bày tỏ mong muốn đóng góp vào thành công của Hội nghị WEF ASEAN tại Việt 
Nam, hỗ trợ Việt Nam nâng cao năng suất lao động, chia sẻ kinh nghiệm trong xây dựng Chính phủ điện tử và
ủng hộ quan điểm của Việt Nam và các nước ASEAN trong vấn đề Biển Đông.

Text:
{text}

Summary:

"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=1000,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(f'INPUT PROMPT:\n{prompt}')
print(output)

INPUT PROMPT:

Summarize the following text:

Text:
Thủ tướng Nguyễn Xuân Phúc và Bộ trưởng Ngoại giao Nhật Bản Taro Kono - Ảnh: VGP/Quang Hiếu. Tại buổi tiếp,
Thủ tướng Nguyễn Xuân Phúc bày tỏ vui mừng trước sự phát triển nhanh chóng, thực chất và toàn diện của quan
hệ Đối tác chiến lược sâu rộng Việt Nam - Nhật Bản; cho rằng việc hai nước đẩy mạnh trao đổi và tiếp xúc 
đoàn cấp cao thời gian qua đã giúp tăng cường sự tin cậy- điều kiện quan trọng để thúc đẩy quan hệ Việt 
Nam - Nhật Bản phát triển hơn nữa trên mọi lĩnh vực; mong muốn tăng cường hợp tác ASEAN - Nhật Bản và 
Mekong - Nhật Bản, nhất là khi Việt Nam đảm nhiệm vai trò Điều phối viên quan hệ ASEAN - Nhật Bản giai 
đoạn 2018 - 2021; đánh giá cao Nhật Bản đã cung cấp vốn hỗ trợ phát triển chính thức ODA cho Việt Nam thời
gian qua, khẳng định Chính phủ Việt Nam coi trọng và nỗ lực thực hiện hiệu quả các dự án vốn vay ODA giữa
hai nước. Ảnh: VGP/Quang Hiếu. Thủ tướng Nguyễn Xuân Phúc đề nghị Bộ trưởng Ngoại giao Nhật Bản quan 

# 4. Fine tuning using LoRa

## 4.1 Preprocess dataset

In [10]:
def tokenize_function(example):
    prompt_template = """
    Summarize the following text:
    Text: 
    {text}
    
    Summary:
    """    
    prompt = [prompt_template.format(text = text) for text in example["Original Text"]]
    # Explicitly set max_length to handle longer sequences
    example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, max_length=560, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["Summary"], padding='max_length', truncation=True, max_length=200, return_tensors="pt").input_ids


    return example

# The dataset actually contains 3 different splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['Original Text', 'Summary'])

Map:   0%|          | 0/9943 [00:00<?, ? examples/s]

In [11]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(7500))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(7500,9200))
test_dataset = tokenized_datasets["train"].select(range(9200,-1))

In [12]:
a = [len(x['input_ids']) for x in tokenized_datasets['train']]

## 4.2 Set up LoRa

In [13]:
from peft import LoraConfig, get_peft_model, TaskType

In [14]:
LORA_R = 16  
LORA_ALPHA = 16
LORA_DROPOUT = 0.1
TARGET_MODULES = ["q", "v"]

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

In [15]:
peft_model = get_peft_model(original_model, config)
number_of_parameters(peft_model)

Số lượng tham số trong mô hình: 61096448
Số lượng tham số có thể huấn luyện: 589824
Phần trăm tham số huấn luyện: 0.9653981848502878%


## 4.3 Train

In [17]:
output_dir = f'./peft-summarize-text-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size = False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-4,
    weight_decay=0.01,
    eval_steps=100,  # Evaluation frequency
    save_steps=500,  # Save checkpoint every 500 steps
    num_train_epochs=5,  
    logging_steps=1,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

In [18]:
peft_trainer.train()

peft_model_path="./peft-summarize-text-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,10.942600
2,10.660500
3,11.186400
4,12.520400
5,8.866300
6,12.807400
7,9.363100
8,9.743500
9,8.079600
10,7.765000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


('./peft-extract-tagname-checkpoint-local/tokenizer_config.json',
 './peft-extract-tagname-checkpoint-local/special_tokens_map.json',
 './peft-extract-tagname-checkpoint-local/tokenizer.json')

In [19]:
peft_trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.39631038904190063,
 'eval_runtime': 24.42,
 'eval_samples_per_second': 69.615,
 'eval_steps_per_second': 2.211,
 'epoch': 5.0}

In [20]:
# peft_trainer.push_to_hub()

In [20]:
from peft import PeftModel, PeftConfig

tokenizer = AutoTokenizer.from_pretrained("pengold/t5-vietnamese-summarization")
original_model = AutoModelForSeq2SeqLM.from_pretrained("pengold/t5-vietnamese-summarization")
peft_model = PeftModel.from_pretrained(original_model,
                                       './peft-extract-tagname-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [25]:
print(number_of_parameters(peft_model))

Số lượng tham số trong mô hình: 300864896
Số lượng tham số có thể huấn luyện: 0
Phần trăm tham số huấn luyện: 0.0%
None


In [24]:
text = """
Báo Indonesia: "Tuyển Việt Nam mạnh nhất Đông Nam Á, là ngựa ô ở Asian Cup" (Dân trí) - Tờ Bola.Okezone (Indonesia) đánh giá rất cao sức mạnh của đội tuyển Việt Nam. Họ xếp đoàn quân của HLV Troussier vào nhóm ngựa ô (có thể gây bất ngờ) ở Asian Cup 2023. Trong bài viết vào sáng nay (9/12), tờ Bola.Okezone đã chỉ ra 5 đội trong nhóm ngựa ô ở Asian Cup 2023. Đội tuyển Việt Nam có mặt trong danh sách này cùng với Tajikistan, Malaysia, Thái Lan và Indonesia. Tờ Bola.Okezone đánh giá đội tuyển Việt Nam mạnh nhất Đông Nam Á (Ảnh: Minh Quân). Nói về sức mạnh của đoàn quân HLV Troussier, tờ báo của Indonesia bình luận: "Đội tuyển Việt Nam đủ điều kiện tham dự Asian Cup 2023 nhờ lọt vào vòng loại thứ ba World Cup 2022. Có thể nói, đội bóng của HLV Troussier mạnh nhất Đông Nam Á thời điểm này. Không ngạc nhiên nếu như đội tuyển Việt Nam có thể tạo nên bất ngờ ở Asian Cup 2023". Bình luận về đội tuyển Indonesia, tờ Bola.Okezone viết: "Đội tuyển Indonesia hoàn toàn có thể tạo nên bất ngờ ở Asian Cup 2023. Với sự góp mặt của nhiều cầu thủ nhập tịch, HLV Shin Tae Yong có thể biến đội bóng trở thành tập thể tốt ở châu Á. Với việc cầu thủ đang thi đấu ở giải Ngoại hạng Anh, Justin Hubner, nhập tịch thành công, sức mạnh của Indonesia sẽ được cải thiện đáng kể. Trước đây, đội tuyển xứ Vạn đảo có 4 lần tham dự Asian Cup nhưng chưa từng vượt qua vòng bảng. Tuy nhiên, lần này, họ quyết định phá dớp buồn". Đánh giá về sức mạnh của Thái Lan, tờ báo của Indonesia bình luận: "Thái Lan từng giành vị trí thứ 3 chung cuộc tại Asian Cup 1972 nhưng sau đó họ chưa có lần nào lọt vào bán kết. Ở thời điểm này, Thái Lan vừa chia tay HLV Mano Polking. Tuy nhiên, họ sẽ có thể tạo ra ngạc nhiên cho đối thủ". Báo Indonesia đánh giá cao sức mạnh của đội nhà (Ảnh: PSSI). Tờ Bola.Okezone cũng đánh giá cao Malaysia: "Malaysia cũng như Indonesia. Họ có 4 lần tham dự Asian Cup nhưng đều dừng bước từ vòng bảng. Dù vậy, đội bóng này đã thay da đổi thịt dưới thời HLV Kim Pan Gon. Tại vòng loại World Cup 2026, Malaysia giữ thành tích toàn thắng khi hạ gục Kyrgyzstan và Đài Bắc Trung Hoa". Cuối cùng, tờ báo xứ Vạn đảo viết về Tajikistan: "Họ chỉ là tân binh của Asian Cup nhưng màn trình diễn ở vòng loại cho thấy Tajikistan là đội bóng nhiều hứa hẹn. Tại vòng loại, Tajikistan đã đứng đầu bảng khi đối đầu với Kyrgyzstan, Singapore và Myanmar". Ở Asian Cup 2023, đội tuyển Việt Nam sẽ lần lượt chạm trán với Nhật Bản (18h30 ngày 14/1/2024), Indonesia (21h30 ngày 19/1), Iraq (18h30 ngày 24/1).
"""

In [25]:
prompt = f"""
Summarize the following text:
Text: 
{text}

Summary:
"""

In [29]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=1000, num_beans = 1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
print(peft_model_text_output)

Một HLV Troussier vào nhóm ngựa ô (có thể gây bất ngờ) ở Asian Cup 2023. Đội tuyển Việt Nam mạnh nhất Đông Nam Á thời điểm này.


# 5. Evaluate the Model

In [28]:
rouge = evaluate.load('rouge')

In [39]:
texts = dataset['train'][9200:9210]['Original Text']
human_baseline_summaries = dataset['train'][9200:9210]['Summary']

original_model_summaries = []
instruct_model_summaries = []

for _, text in enumerate(texts):
    prompt = f"""
    Summarize the following text:
    Text: 
    {text}

    Summary:
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids


    instruct_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'instruct_model_summaries'])
df.head()

,human_baseline_summaries,instruct_model_summaries
0,Tờ báo thể thao hàng đầu Indonesia khen ngợi m...,Trong cuộc chiến giành vé chung kết AFF Cup 20...
1,HLV Park Hang Seo tỏ ra khá bất ngờ trước phát...,HLV Park Hang Seo về việc tuyển Việt Nam mạnh ...
2,"HLV Park Hang Seo đã nỗ lực, tăng cường cho qu...","HLV Park Hang Seo, nhưng rốt cuộc vì... quân d..."
3,Đặng Văn Lâm lập kỷ lục giữ sạch lưới sau trận...,Đặng Văn Lâm vừa vừa công bố các trận hoà của ...
4,"Trong thời gian gần đây, MU đang xem Goncalo R...","Trong thời gian gần đây, Mudryk, Bayern Munich..."


In [40]:
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('INSTRUCT MODEL:')
print(instruct_model_results)

INSTRUCT MODEL:
{'rouge1': 0.4780357899093211, 'rouge2': 0.31586600784034924, 'rougeL': 0.3728480668662625, 'rougeLsum': 0.3743512510897763}
